# Citibike ML
In this example we use the [Citibike dataset](https://ride.citibikenyc.com/system-data). Citibike is a bicycle sharing system in New York City. Everyday users choose from 20,000 bicycles at 1300 stations around New York City.

To ensure customer satisfaction Citibike needs to predict how many bicycles will be needed at each station. Maintenance teams from Citibike will check each station and repair or replace bicycles. Additionally, the team will relocate bicycles between stations based on predicted demand. The business needs to be able to run reports of how many bicycles will be needed at a given station on a given day.

## ML Engineering Development
In this section of the demo, we will utilize Snowpark's Python client-side Dataframe API to build and develope code for the **ML Ops pipeline**.  We will take the functions and model training/inference definition from the data scientist and put it into production using the Snowpark server-side runtime and Snowpark Python user-defined functions for ML model training and inference.

The ML Engineer will start by exploring the deoployment options and testing the deployed model before building a pipeline.

For this demo flow we will assume that the organization has the following **policies and processes** :   
-**Dev Tools**: The ML engineer can develop in their tool of choice (ie. VS Code, IntelliJ, Pycharm, Eclipse, etc.).  Snowpark Python makes it possible to use any environment where they have a python kernel.  For the sake of a demo we will use Jupyter.  
-**Data Governance**: To preserve customer privacy no data can be stored locally.  The ingest system may store data temporarily but it must be assumed that, in production, the ingest system will not preserve intermediate data products between runs. Snowpark Python allows the user to push-down all operations to Snowflake and bring the code to the data.   
-**Automation**: Although the ML engineer can use any IDE or notebooks for development purposes the final product must be python code at the end of the work stream.  Well-documented, modularized code is necessary for good ML operations and to interface with the company's CI/CD and orchestration tools.  
-**Compliance**: Any ML models must be traceable back to the original data set used for training.  The business needs to be able to easily remove specific user data from training datasets and retrain models. 

Input: Data in `trips` table.  Feature engineering, train, predict functions from data scientist.  
Output: Prediction models available to business users in SQL. Evaluation reports for monitoring.

### 1. Load  credentials and connect to Snowflake

In [ ]:
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect('./include/state.json')

### 2.  Create Feature Pipelines


In [ ]:
trips_table_name = state_dict['trips_table_name']
holiday_table_name = state_dict['holiday_table_name']
weather_table_name = state_dict['weather_table_name']

We will materialize the holiday and weather datasets as tables instead of calculating each time in the inference and training pipelines.

In [ ]:
import snowflake.snowpark as snp
from snowflake.snowpark import functions as F 
from dags.feature_engineering import generate_holiday_df, generate_weather_df

holiday_df = generate_holiday_df(session, holiday_table_name) 
holiday_df.write.mode('overwrite').saveAsTable(holiday_table_name)

weather_df = generate_weather_df(session, weather_table_name) 
weather_df.write.mode('overwrite').saveAsTable(weather_table_name)


We will need to forecast features for weather (TEMP) and also holidays.  For weather we could use forecast from the Snowflake Marketplace providers like Weather Source.  Since we went back in time for this hands-on-lab the weather "forecast" is actually in the historical weather tables.

In [ ]:
def create_forecast_df(session, holiday_table_name:str, weather_table_name:str, start_date:str, steps:int):
    from dags.feature_engineering import generate_holiday_df, generate_weather_df
    from datetime import timedelta, datetime
    #import snowflake.snowpark as snp
    from snowflake.snowpark import functions as F 
    
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = start_date+timedelta(days=steps)

    #check if it tables already materialized, otherwise generate DF
    holiday_df = session.table(holiday_table_name)
    try: 
        _ = holiday_df.columns()
    except:
        holiday_df = generate_holiday_df(session, holiday_table_name)
        
    weather_df = session.table(weather_table_name)
    try: 
        _ = weather_df.columns()
    except:
        weather_df = generate_weather_df(session, weather_table_name)
        
    forecast_df = holiday_df.join(weather_df[['DATE','TEMP']], 'DATE', join_type='right')\
                            .na.fill({'HOLIDAY':0})\
                            .filter((F.col('DATE') >= start_date) &\
                                    (F.col('DATE') <= end_date))\
                            .sort('DATE', ascending=True)
    return forecast_df

In [ ]:
forecast_df = create_forecast_df(session=session, 
                                 holiday_table_name=state_dict['holiday_table_name'], 
                                 weather_table_name=state_dict['weather_table_name'], 
                                 start_date='2020-03-01', 
                                 steps=30)
forecast_df.show()

#### Vectorized feature generation
Previously the data scientist picked one station for training and predictions.  We want to generate features for all stations in parallel.  We can leverage the power of the Snowflake SQL execution engine for this and Snowpark allows us to write it in python.  

Snowflake [window functions](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/_autosummary/snowflake.snowpark.html#snowflake.snowpark.Window) are a powerful tool for vectorizing work.  Our initial feature engineering code from the data scientist used window functions to calculate the lag features.

We can create multi-level window functions to partition by station_id and then group by date within that window.  
  
Also we will create a second window in order to make sure that each station of our features have at least 2 years worth of data (needed for the model to pickup seasonality) and has activity up to the latest date of the current dataset (to build forecast).

In [ ]:
%%time
sid_date_window = snp.Window.partition_by(F.col('STATION_ID')).order_by(F.col('DATE').asc())
sid_window = snp.Window.partition_by(F.col('STATION_ID'))
latest_date = session.table(trips_table_name).select(F.to_char(F.to_date(F.max('STARTTIME')))).collect()[0][0]

historical_df = session.table(trips_table_name)\
                       .select(F.to_date(F.col('STARTTIME')).alias('DATE'),
                               F.col('START_STATION_ID').alias('STATION_ID'))\
                       .group_by(F.col('STATION_ID'), F.col('DATE'))\
                            .count()\
                       .with_column('LAG_1', F.lag(F.col('COUNT'), offset=1).over(sid_date_window))\
                       .with_column('LAG_7', F.lag(F.col('COUNT'), offset=7).over(sid_date_window))\
                       .with_column('LAG_365', F.lag(F.col('COUNT'), offset=365).over(sid_date_window))\
                            .na.drop()\
                       .join(holiday_df, 'DATE', join_type='left').na.fill({'HOLIDAY':0})\
                       .join(weather_df[['DATE','TEMP']], 'DATE', 'inner')\
                       .with_column('DAY_COUNT', F.count(F.col('DATE')).over(sid_window))\
                            .filter(F.col('DAY_COUNT') >= 365*2)\
                       .with_column('MAX_DATE', F.max('DATE').over(sid_window))\
                            .filter(F.col('MAX_DATE') == latest_date)

Make sure all stations have data upto the latest date.

In [ ]:
historical_df.group_by('STATION_ID').max('DATE').select(F.min(F.col('MAX(DATE)'))).show()

Our feature set should not include any stations with less than 730 days (365*2) of data.

In [ ]:
historical_df.select(F.min('DAY_COUNT')).collect()[0][0]

We'll drop the day count and max date columns since we don't really need them beyond verification.

In [ ]:
historical_df = historical_df.drop(['DAY_COUNT', 'MAX_DATE'])

Now how many stations have at least two years of data with recent activity?

In [ ]:
historical_df.select('STATION_ID').distinct().count()

In [ ]:
# historical_df = historical_df.drop('DAY_COUNT')

# historical_column_list = historical_df.columns
# historical_column_list.remove('STATION_ID')
# historical_column_names = F.array_construct(*[F.lit(x) for x in historical_column_list])

# historical_df_stuffed = historical_df.group_by(F.col('STATION_ID'))\
#                                      .agg(F.array_agg(F.array_construct(*historical_column_list))\
#                                           .alias('HISTORICAL_DATA'))

In [ ]:
#historical_df_stuffed.filter(F.col('STATION_ID') == '519').show()

### 3. Create UDF for Training and Inference

Since this is a time series prediction we will retrain a model each time we do inference.  We don't need to save the model artefacts but we will save the predictions in an predictions table.  
  
Here we can use Snowpark User Defined Functions for training as well as inference without having to pull data out of Snowflake.

In [ ]:
%%writefile dags/station_train_predict.py

def station_train_predict_func(historical_data:list, 
                               historical_column_names:list, 
                               target_column:str,
                               cutpoint: int, 
                               max_epochs: int, 
                               forecast_data:list,
                               forecast_column_names:list,
                               lag_values:list) -> str:
    
    from torch import tensor
    import pandas as pd
    from pytorch_tabnet.tab_model import TabNetRegressor
    from datetime import timedelta
    import numpy as np
    
    feature_columns = historical_column_names.copy()
    feature_columns.remove('DATE')
    #feature_columns.remove('STATION_ID')
    feature_columns.remove(target_column)
    
    df = pd.DataFrame(historical_data, columns = historical_column_names)
    
    ##In order to do train/valid split on time-based portion the input data must be sorted by date    
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.sort_values(by='DATE', ascending=True)
    
    y_valid = df[target_column][-cutpoint:].values.reshape(-1, 1)
    X_valid = df[feature_columns][-cutpoint:].values
    y_train = df[target_column][:-cutpoint].values.reshape(-1, 1)
    X_train = df[feature_columns][:-cutpoint].values
    
    model = TabNetRegressor()

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        max_epochs=max_epochs,
        patience=100,
        batch_size=128, 
        virtual_batch_size=64,
        num_workers=0,
        drop_last=True)
    
    df['PRED'] = model.predict(tensor(df[feature_columns].values))

    #Now make the multi-step forecast
    if len(lag_values) > 0:
        forecast_df = pd.DataFrame(forecast_data, columns = forecast_column_names)
        
        for step in range(len(forecast_df)):
            #station_id = df.iloc[-1]['STATION_ID']
            future_date = df.iloc[-1]['DATE']+timedelta(days=1)
            lags=[df.shift(lag-1).iloc[-1]['COUNT'] for lag in lag_values]
            forecast=forecast_df.loc[forecast_df['DATE']==future_date.strftime('%Y-%m-%d')]
            forecast=forecast.drop(labels='DATE', axis=1).values.tolist()[0]
            features=[*lags, *forecast]
            pred=round(model.predict(np.array([features]))[0][0])
            #row=[future_date, station_id, pred, *features, pred]
            row=[future_date, pred, *features, pred]
            df.loc[len(df)]=row
    
    explain_df = pd.DataFrame(model.explain(df[feature_columns].astype(float).values)[0], 
                         columns = feature_columns).add_prefix('EXPL_').round(2)
    df = pd.concat([df.set_index('DATE').reset_index(), explain_df], axis=1)
    df['DATE'] = df['DATE'].dt.strftime('%Y-%m-%d')
    
    return [df.values.tolist(), df.columns.tolist()]



The Snowpark server-side Anaconda runtime has a large [list of Python modules included](https://docs.snowflake.com/en/LIMITEDACCESS/udf-python-packages.html#list-of-the-third-party-packages-from-anaconda) for our UDF.  However, the data scientist built this code based on pytorch-tabnet which is not currently in the Snowpark distribution.
  
  We can simply add [pytorch_tabnet](https://github.com/dreamquark-ai/tabnet), as well as our own team's python code, as import dependencies.

In [ ]:
from dags.station_train_predict import station_train_predict_func
import os 

#We can add dependencies from locally installed directories
#source_dir = os.environ['CONDA_PREFIX']+'/lib/python3.8/site-packages/'

model_stage_name = 'model_stage'
_ = session.sql('CREATE STAGE IF NOT EXISTS '+str(model_stage_name)).collect()

session.clear_packages()
session.clear_imports()
dep_packages=["pandas==1.3.5", "pytorch", "scipy", "scikit-learn"]
dep_imports=['./include/pytorch_tabnet.zip', 'dags']

station_train_predict_udf = session.udf.register(station_train_predict_func, 
                                                 name="station_train_predict_udf",
                                                 is_permanent=True,
                                                 stage_location='@'+str(model_stage_name), 
                                                 imports=dep_imports,
                                                 packages=dep_packages,
                                                 replace=True)

#### Vectorize the training and inference

Because we currently only have scalar functions for Snowpark Python UDFs we must aggregate the features for each station to a single cell.   This will be much easier in the future with vectorized input and user-defined table functions.

In [ ]:
historical_column_list = historical_df.columns
historical_column_list.remove('STATION_ID')
historical_column_names = F.array_construct(*[F.lit(x) for x in historical_column_list])

historical_df = historical_df.group_by(F.col('STATION_ID'))\
                             .agg(F.array_agg(F.array_construct(*historical_column_list))\
                                  .alias('HISTORICAL_DATA'))

forecast_column_names = F.array_construct(*[F.lit(x) for x in forecast_df.columns])
forecast_df = forecast_df.select(F.array_agg(F.array_construct(F.col('*'))).alias('FORECAST_DATA'))

### 4. Test the training/inference pipeline and prediction output.

With the array-stuffed dataframes we can now call the UDF for training and predictions.

In [ ]:
session.use_warehouse(state_dict['compute_parameters']['train_warehouse'])

In [ ]:
%%time
cutpoint=365
max_epochs = 10
target_column = 'COUNT'
lag_values=[1,7,365]
lag_values_array = F.array_construct(*[F.lit(x) for x in lag_values])

historical_df.join(forecast_df)\
             .select(F.col('STATION_ID'),
                     F.call_udf('station_train_predict_udf', 
                                F.col('HISTORICAL_DATA'),
                                F.lit(historical_column_names), 
                                F.lit(target_column),
                                F.lit(cutpoint), 
                                F.lit(max_epochs),
                                F.col('FORECAST_DATA'),
                                F.lit(forecast_column_names),
                                F.lit(lag_values_array)).alias('OUTPUT_DATA'))\
             .write.mode('overwrite')\
             .save_as_table('PRED_test')

In [ ]:
pred_df = session.table('PRED_test')

Again, because we don't yet have UDTFs we need to read the output to a json object and parse it locally.

In [ ]:
import ast
import pandas as pd
row=10
df = pd.DataFrame(ast.literal_eval(output_list[row]['OUTPUT_DATA'])[0], 
                  columns = ast.literal_eval(output_list[row]['OUTPUT_DATA'])[1])

df['STATION_ID']=output_list[row]['STATION_ID']
df.tail()

In [ ]:
def plot(df, x_lab:str, y_true_lab:str, y_pred_lab:str):
    plt.figure(figsize=(15, 8))
    df = pd.melt(df, id_vars=[x_lab], value_vars=[y_true_lab, y_pred_lab])
    ax = sns.lineplot(x=x_lab, y='value', hue='variable', data=df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
   
plot(df[-365:], 'DATE', 'COUNT', 'PRED')

We will end by consolidating the functions we created.

In [ ]:
#%%writefile citibike_ml/mlops_pipeline.py

def materialize_holiday_table(session, holiday_table_name:str) -> str:
    from dags.feature_engineering import generate_holiday_df
    
    holiday_df = generate_holiday_df(session=session, holiday_table_name=holiday_table_name)
    holiday_df.write.mode('overwrite').saveAsTable(holiday_table_name)
    
    return holiday_table_name

def materialize_weather_table(session, weather_table_name:str) -> str:
    from dags.feature_engineering import generate_weather_df

    weather_df = generate_weather_df(session=session, weather_table_name=weather_table_name)
    weather_df.write.mode('overwrite').saveAsTable(weather_table_name)
    
    return weather_table_name

def deploy_pred_train_udf(session, function_name, model_stage_name) -> str:
    from dags.station_train_predict import station_train_predict_func

    session.clear_packages()
    session.clear_imports()

    
    
    

    station_train_predict_udf = session.udf.register(station_train_predict_func, 
                                                  name=function_name,
                                                  is_permanent=True,
                                                  stage_location='@'+str(model_stage_name), 
                                                  replace=True)
    return station_train_predict_udf.name

def create_forecast_df(session, holiday_table_name:str, weather_table_name:str, start_date:str, steps:int):
    from dags.feature_engineering import generate_holiday_df, generate_weather_df
    from datetime import timedelta, datetime
    #import snowflake.snowpark as snp
    from snowflake.snowpark import functions as F 
    
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = start_date+timedelta(days=steps)

    #check if it tables already materialized, otherwise generate DF
    holiday_df = session.table(holiday_table_name)
    try: 
        _ = holiday_df.columns()
    except:
        holiday_df = generate_holiday_df(session, holiday_table_name)
        
    weather_df = session.table(weather_table_name)[['DATE','TEMP']]
    try: 
        _ = weather_df.columns()
    except:
        weather_df = generate_weather_df(session, weather_table_name)[['DATE','TEMP']]
        
    forecast_df = holiday_df.join(weather_df, 'DATE', join_type='right')\
                            .na.fill({'HOLIDAY':0})\
                            .filter((F.col('DATE') >= start_date) &\
                                    (F.col('DATE') <= end_date))\
                            .sort('DATE', ascending=True)
    return forecast_df


def generate_feature_table(session, 
                           clone_table_name, 
                           feature_table_name, 
                           holiday_table_name, 
                           weather_table_name) -> list:

    import snowflake.snowpark as snp
    from snowflake.snowpark import functions as F 
    
    #start_date, end_date = input_df.select(F.min('STARTTIME'), F.max('STARTTIME')).collect()[0][0:2]
    
    #check if features are already materialized (or in a temp table)
    holiday_df = session.table(holiday_table_name)
    try: 
        _ = holiday_df.columns()
    except:
        holiday_df = generate_holiday_df(session, holiday_table_name)
        
    weather_df = session.table(weather_table_name)[['DATE','TEMP']]
    try: 
        _ = weather_df.columns()
    except:
        weather_df = generate_weather_df(session, weather_table_name)[['DATE','TEMP']]
        
        
        
        
        
        
        
        
        
    return junk





In [ ]:
#session.close()